In [11]:
import cv2
import matplotlib.pyplot as plt
import torch
print(torch.__version__)

from PIL import Image
import numpy as np


2.5.1


In [13]:
import torchvision.transforms as T
import torchvision

AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [14]:
from transformers import pipeline

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)